#### Agents are one of the newest and most experimental parts of Langchain, but they offer huge potential for large language model based applications by combining what we've already learned about Model IO, Data Connections and Chains, we've actually already approached applications that we've built ourselves that are similar to agents allowing us to create more robust applications.

https://python.langchain.com/docs/modules/agents/

----

***However, agents take this one step further with the ReAct framework.***

At their core, agents allow large language models to connect to tools like using Wikipedia, a calculator, Google search and deciding between those tools, and then conduct a structured approach to complete a task.

It's based on a paper called `'ReAct: Synergising Reasoning and Acting in Language Models'`.

PDF: https://arxiv.org/pdf/2210.03629.pdf

What this paper describes is that a lot of other applications focused on
- Either just reasoning, so we have our language model, try to trace the reasoning for the correct answer.
- Or we had models that were just able to act.
    
    For example: They were able to take an action on some sort of environment and then get some observation from that.

The react prompting tries to combine these so you have reasoning traces to your large language models that then inform what actions it should take, and then it observes the results of those actions.

-----
-----

In **Langchain implementation**, this takes the form of an agent object that has access to a list of tools.

The agent is given a task and can reason which tools are appropriate to use and can then utilize those results or observations to continue via an internal chain until it actually solves the task that you initially asked it.


Now, internally, Langchain labels the thinking in terms of an observation, a thought, and then an action.
- And then the cycle repeats itself.

So the large language model is going to have some sort of observation, it's going to think about it and then perform an action and based off the results of the action, it does another observation and continues that cycle until it feels that it solved the problem or task we gave it.

In [ ]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.1.1
    Uninstalling pydantic-2.1.1:
      Successfully uninstalled pydantic-2.1.1


In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [ ]:
api_key = open("/content/api_key.txt").read()
llm = ChatOpenAI(openai_api_key= api_key, temperature=0)

In [ ]:
# Step2: setting up our tools and connecting it to the LLM
tools = load_tools(['llm-math'], llm = llm)

In [ ]:
type(tools)

list

In [ ]:
#step3: Choose Agent and Initialize Agent type
dir(AgentType)

['CHAT_CONVERSATIONAL_REACT_DESCRIPTION',
 'CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'CONVERSATIONAL_REACT_DESCRIPTION',
 'OPENAI_FUNCTIONS',
 'OPENAI_MULTI_FUNCTIONS',
 'REACT_DOCSTORE',
 'SELF_ASK_WITH_SEARCH',
 'STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION',
 'ZERO_SHOT_REACT_DESCRIPTION',
 '__class__',
 '__doc__',
 '__members__',
 '__module__']

In [ ]:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True)

In [ ]:
agent.run("What is 134292 times 29384920?")



> Entering new AgentExecutor chain...
I need to multiply 134292 by 29384920.
Action: Calculator
Action Input: 134292 * 29384920
Observation: Answer: 3946159676640
Thought:I now know the final answer
Final Answer: 3946159676640

> Finished chain.


'3946159676640'

What's going on inside the chain?

When we run this:
- It enters a new chain.

- So it has this thought that "*I need to multiply these two numbers*". Here, it called a large language model and asked it like, What's my task here. And then it thinks, okay, that's my task.

- Then it sees the list of tools available. In this case, there's basically just two tools its own internal large language model like ChatGPT, and then the ability to calculate something.

- So it realizes the calculator is the correct tool here and that calculator is given to it via the 'llm_math' call.
(Later, we talk about other integrations and agent tools, we'll see it be able to distinguish between doing something like a Google search and a calculator.)

- So it figured out, okay, the '`Action`' I need is the calculator.

- Then most of these tools, pretty much all of them need some sort of input for the action. In this case, it realizes that the correct `input` for a calculator is the formula, so it passes in the formula.

- Then the calculator responds back using an eval call internally with the answer which is also the `Observation` of the action result.

- Then it `thinks` okay does based off the original LlM and the fact that you need to multiply this, does this look correct to the large language model?

- Large language model thinks Yeah, that looks correct "I know the `Final Answer`, and then it just outputs the number.

In [ ]:
# Making it user facing:
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = False)

In [ ]:
agent.run("What is 134292 times 29384920?")

'3946159676640'

# Agent Tools

https://python.langchain.com/docs/integrations/tools/serpapi

In [ ]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI

In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=9d5c815ff10a6d9bcd060c7f1beb44d07e96c6c3e97c6a11bf5320b4f869121a
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results


In [ ]:
# API --> https://serpapi.com/users/welcome
serp_api_key = open("/content/serp_spi_key.txt").read()

In [ ]:
import os
os.environ['SERPAPI_API_KEY'] = serp_api_key

In [ ]:
tools = load_tools(['serpapi', 'llm-math'], llm=llm)

In [ ]:
agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                          verbose = True)

In [ ]:
agent.run("What year was Mahatma Gandhi born? What is that year multiplies by 5?")



> Entering new AgentExecutor chain...
I can use the search tool to find out the year Mahatma Gandhi was born. Then I can use the calculator to multiply that year by 5.
Action: Search
Action Input: "year Mahatma Gandhi was born"
Observation: October 2, 1869
Thought:Now I can use the calculator to multiply 1869 by 5.
Action: Calculator
Action Input: 1869 * 5
Observation: Answer: 9345
Thought:I now know the final answer
Final Answer: The year Mahatma Gandhi was born is 1869 and when multiplied by 5, it equals 9345.

> Finished chain.


'The year Mahatma Gandhi was born is 1869 and when multiplied by 5, it equals 9345.'

### Python Agent Tool:
More specialize agent tool that allows the agent to actually run and execute Python code.

In [ ]:
from langchain.tools.python.tool import PythonREPLTool

In [ ]:
from langchain.python import PythonREPL

In [ ]:
from langchain.agents.agent_toolkits import create_python_agent

In [ ]:
agent = create_python_agent(tool = PythonREPLTool(),
                            llm = llm,
                            agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                            verbose = True)

In [ ]:
python_list = [1,2,33,4,5,6,3,5,3,2,3,5,2,3,34,2,6,56,2]

In [ ]:
agent.run(f'''Sort this Python list:{python_list}''')



> Entering new AgentExecutor chain...


I can use the `sorted()` function to sort the list in ascending order.
Action: Python_REPL
Action Input: sorted([1, 2, 33, 4, 5, 6, 3, 5, 3, 2, 3, 5, 2, 3, 34, 2, 6, 56, 2])
Observation: 
Thought:The list has been sorted in ascending order.
Final Answer: [1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6, 33, 34, 56]

> Finished chain.


'[1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6, 33, 34, 56]'

# Custom Tools

In [ ]:
# tool is a decorator
from langchain.agents import tool

In [ ]:
# LLM -> tools --> Docstring --> pick tool --> Action --> Observation
@tool
def coolest_guy(text: str) -> str:

  '''Returns the name of the coolest person in the universe. \
  Expects an input of an empty string '' \
  and returns the name of the coolest person in the universe.'''

  return "Gaurav Dahal"

To convert this function into tool we just need to decorate it with `@tool` at the top of the function.

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=aea985a3403e45d4d971fcaab33673292a01ba2158b3f464be57fc3c7ba0514f
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
tools = load_tools(['wikipedia', 'llm-math'], llm = llm)

In [ ]:
agent = initialize_agent(tools, llm,
                         agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True)

In [ ]:
agent.run("Who is the coolest person in the universe?")



> Entering new AgentExecutor chain...
This question is subjective and cannot be answered with factual information.
Final Answer: There is no definitive answer to this question as it is based on personal opinion.

> Finished chain.


'There is no definitive answer to this question as it is based on personal opinion.'

Overriding my model with my own tool

In [ ]:
# we can concatenate lists together
tools = load_tools(['wikipedia', 'llm-math'], llm = llm)
# here, I'm passing in the function as an item in a list,
# but I am not actually executing the funcction
tools = tools + [coolest_guy]

In [ ]:
type(tools)

list

In [ ]:
agent = initialize_agent(tools, llm, agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True)

In [ ]:
agent.run("Who is the coolest person in the universe?")



> Entering new AgentExecutor chain...
I can use the coolest_guy tool to find the answer.
Action: coolest_guy
Action Input: ''
Observation: Gaurav Dahal
Thought:I now know the final answer
Final Answer: Gaurav Dahal

> Finished chain.


'Gaurav Dahal'

You can perform API calls to any API you can think of.

- say *@tool*,
- make some sort of API call like, pass in text, string etc.
- then internally you would just describe either your own company's API, any sort of API you want. So you'll say *"Connect to this API, use this key."* etc...
- then, just return the answer eventually, based off what you want it to do.

In [ ]:
@tool
def some_api(text: str) -> str:
  '''Connect this API use this key'''
  return answer

In [ ]:
agent.run("What time is it right now?")



> Entering new AgentExecutor chain...


OutputParserException: ignored

Agents are pretty experimental. Don't be surprised if you get these sort of parsing errors as you continue to work with agents yourself.

### How do we fix this error?

Let's make a tool that allows a large language model to access the current date based off my computer or the current time based off my computer.

In [ ]:
from datetime import datetime
# Changing into string so that the model can actually read
str(datetime.now())

'2023-08-16 09:16:40.915566'

In [ ]:
# I just need to create this into a function and then call it via tool

@tool
def get_current_time(text: str) -> str:
  ''' Returns this current time. Use this for any question \
  regarding the current time. Input is an empty string and the current \
  time is returned in a string format. \
   Only use this funtion for the current time, other time related questions \
   should use another tool.'''
  return str(datetime.now())

In [ ]:
agent = initialize_agent(tools + [get_current_time],
                         llm,
                         agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose = True)

In [ ]:
agent.run('What is the current time?')



> Entering new AgentExecutor chain...
I should use the get_current_time tool to get the current time.
Action: get_current_time
Action Input: ''
Observation: 2023-08-16 09:22:29.290380
Thought:I now know the final answer
Final Answer: The current time is 2023-08-16 09:22:29.290380.

> Finished chain.


'The current time is 2023-08-16 09:22:29.290380.'

Now, here is a key question that we should be thinking about.

What happens if I ask it the time of a historical event?

In [ ]:
agent.run("When did India got its Independence?")



> Entering new AgentExecutor chain...
I need to find the date when India gained independence.
Action: Wikipedia
Action Input: "India independence date"
Observation: Page: Independence Day (India)
Summary: Independence Day is celebrated annually on 15 August as a public holiday in India commemorating the nation's independence from the United Kingdom on 15 August 1947, the day when the provisions of the Indian Independence Act, which transferred legislative sovereignty to the Indian Constituent Assembly, came into effect. India retained King George VI as head of state until its transition to a republic, when the Constitution of India came into effect on 26 January 1950 (celebrated as Indian Republic Day) and replaced the dominion prefix, Dominion of India, with the enactment of the sovereign law Constitution of India. India attained independence following the independence movement noted for largely non-violent resistance and civil disobedience led by Indian National Congress under the 

'India gained its independence on 15 August 1947.'

Steps:
1. Create your own function
2. Define what the inputs and outputs should be like
3. When you should use the tool
4. Decorate it and you're good to go

# **Conversation Agents**

Conversation agents allow us to connect what we already know about memory, to keep a running conversation with the idea of an agent that has access to tools.

connecting Agent --> Memory

In [ ]:
from langchain.memory import ConversationBufferMemory

In [ ]:
memory = ConversationBufferMemory(memory_key='chat_history')

In [ ]:
# step1: set up tool and connect to llm
# step2: select and initialize the agent

In [ ]:
tools = load_tools(['llm-math'], llm=llm)

In [ ]:
agent = initialize_agent(tools, llm,
                         agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
                         memory = memory, verbose  = True)

In [ ]:
agent.run(input = "What are some good thai food recipes?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Thai cuisine is known for its bold flavors and aromatic ingredients. Here are a few popular Thai food recipes that you might enjoy:

1. Pad Thai: This stir-fried noodle dish is a classic Thai favorite. It typically includes rice noodles, shrimp or chicken, eggs, tofu, bean sprouts, and a tangy sauce made from tamarind paste, fish sauce, and lime juice.

2. Green Curry: This spicy curry is made with a paste of green chilies, garlic, lemongrass, and other herbs and spices. It is typically cooked with coconut milk, vegetables, and your choice of meat or tofu.

3. Tom Yum Soup: This hot and sour soup is a popular Thai appetizer. It is made with a flavorful broth, shrimp or chicken, mushrooms, lemongrass, lime juice, and chili peppers.

4. Massaman Curry: This rich and flavorful curry is made with a paste of roasted spices, including cinnamon, cardamom, and cloves. It is typically cooked with coconut milk, pota

"Thai cuisine is known for its bold flavors and aromatic ingredients. Here are a few popular Thai food recipes that you might enjoy:\n\n1. Pad Thai: This stir-fried noodle dish is a classic Thai favorite. It typically includes rice noodles, shrimp or chicken, eggs, tofu, bean sprouts, and a tangy sauce made from tamarind paste, fish sauce, and lime juice.\n\n2. Green Curry: This spicy curry is made with a paste of green chilies, garlic, lemongrass, and other herbs and spices. It is typically cooked with coconut milk, vegetables, and your choice of meat or tofu.\n\n3. Tom Yum Soup: This hot and sour soup is a popular Thai appetizer. It is made with a flavorful broth, shrimp or chicken, mushrooms, lemongrass, lime juice, and chili peppers.\n\n4. Massaman Curry: This rich and flavorful curry is made with a paste of roasted spices, including cinnamon, cardamom, and cloves. It is typically cooked with coconut milk, potatoes, onions, and your choice of meat or tofu.\n\n5. Mango Sticky Rice: 

As we're using a memory along with a conversational agent, I can now actually ask it about its previous response.

In [ ]:
agent.run(input = "Which one of those dishes tends to be the spiciest?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Among the dishes mentioned, the spiciest one is typically the Green Curry. Green curry paste is made with green chilies, which gives it its signature heat. However, the level of spiciness can vary depending on personal preference and the amount of chili used in the recipe. If you prefer milder flavors, you can adjust the amount of chili or use a milder variety of chili.

> Finished chain.


'Among the dishes mentioned, the spiciest one is typically the Green Curry. Green curry paste is made with green chilies, which gives it its signature heat. However, the level of spiciness can vary depending on personal preference and the amount of chili used in the recipe. If you prefer milder flavors, you can adjust the amount of chili or use a milder variety of chili.'

In [ ]:
agent.run("Give me the grocery shopping  list for that dish?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Sure! Here's a grocery shopping list for Green Curry:

- Green curry paste
- Coconut milk
- Your choice of meat or tofu (such as chicken, beef, shrimp, or tofu)
- Vegetables (such as bell peppers, eggplant, and bamboo shoots)
- Thai basil leaves
- Fish sauce
- Sugar
- Kaffir lime leaves (optional)
- Thai bird's eye chilies (optional, for extra spiciness)

These are the main ingredients you'll need to make Green Curry. Of course, you can customize the recipe by adding or omitting ingredients based on your preferences. Enjoy cooking! Let me know if there's anything else I can assist you with.

> Finished chain.


"Sure! Here's a grocery shopping list for Green Curry:\n\n- Green curry paste\n- Coconut milk\n- Your choice of meat or tofu (such as chicken, beef, shrimp, or tofu)\n- Vegetables (such as bell peppers, eggplant, and bamboo shoots)\n- Thai basil leaves\n- Fish sauce\n- Sugar\n- Kaffir lime leaves (optional)\n- Thai bird's eye chilies (optional, for extra spiciness)\n\nThese are the main ingredients you'll need to make Green Curry. Of course, you can customize the recipe by adding or omitting ingredients based on your preferences. Enjoy cooking! Let me know if there's anything else I can assist you with."